In [1]:
from utils.dataset import WikiCorpusDataset
print("Categories: ", ", ".join(WikiCorpusDataset().categories))
datasetLoader = WikiCorpusDataset(split_level="sen", category="ALL", display_progress=False)
datasetLoader.create_csv()
datasetLoader.join_csv()

Categories:  BDS, BLD, CLT, EPR, FML, GNM, HST, LTT, PNM, RLW, ROD, SAT, SCL, SNT, TTL
skipping: BDS-sen file already exists!
skipping: BLD-sen file already exists!
skipping: CLT-sen file already exists!
skipping: EPR-sen file already exists!
skipping: FML-sen file already exists!
skipping: GNM-sen file already exists!
skipping: HST-sen file already exists!
skipping: LTT-sen file already exists!
skipping: PNM-sen file already exists!
skipping: RLW-sen file already exists!
skipping: ROD-sen file already exists!
skipping: SAT-sen file already exists!
skipping: SCL-sen file already exists!
skipping: SNT-sen file already exists!
skipping: TTL-sen file already exists!
skipping: sen-ALL file already exists!


In [2]:
from utils.dataset import JESTDataset
JESTDataset().create_csv()

skipped: JEST file already exists!


In [3]:
from utils.dataset import SnowSimplifiedDataset
SnowSimplifiedDataset().create_csv()

skipped: snow_simplified file already exists!


In [4]:
from utils.dataset import TatoebaDataset
TatoebaDataset().create_csv()

skipped: tatoeba file already exists!


In [5]:
from utils.dataset import MassiveTranslationDataset
MassiveTranslationDataset().create_csv()

skipped: massive_translation file already exists!
